# Préparation des données
Dans ce notebook, nous nettoyons les données et ajoutons quelques features de base.

In [1]:
import pandas as pd
import numpy as np
import pickle

## Table Transactions

In [5]:
transactions = pd.read_pickle("pickles/transactions.pkl")

In [7]:
# Transformation des doublons en champ "quantité"
transactions = transactions.groupby(
    transactions.columns.tolist(),
    as_index=False
).size()

transactions.rename({'size': 'quantity'}, axis = 1, inplace = True)

In [8]:
# Tri par date descendante
transactions = transactions.sort_values('t_dat', ascending = False)

In [9]:
# Ajout de la notion de n° de semaine
last_day = transactions['t_dat'].max()
transactions['week_number'] = (last_day - transactions['t_dat']).dt.days // 7

In [13]:
transactions = pd.read_pickle('pickles/transactions_with_week_number.pkl')

In [31]:
# Typage des valeurs
transactions[['sales_channel_id', 'quantity', 'week_number']] = transactions[['sales_channel_id', 'quantity', 'week_number']].astype('int16')
transactions['price'] = transactions['price'].astype('float32')

In [80]:
transactions.to_pickle('pickles/transactions_clean.pkl')
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,quantity,week_number
28813418,2020-09-22,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0898573003,0.033881,2,1,0
28793633,2020-09-22,53d5f95331b01525404c3cbb2da6a84e1173dccb979d28...,0752814021,0.033881,2,2,0
28793635,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,0793506006,0.025407,1,1,0
28793636,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,0802459001,0.008458,1,1,0
28793637,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,0874169001,0.042356,1,1,0


## Table Cart

In [2]:
transactions = pd.read_pickle('pickles/transactions_clean.pkl')

In [3]:
carts = transactions.groupby(["customer_id", "t_dat", "sales_channel_id"], as_index = False).agg(
    total = ("price", "sum"),
    unique_articles = ("article_id", "count"),
    total_articles = ("quantity", "sum"),
    week_number = ("week_number", "max"),
    content = ("article_id", lambda x: list(x))
)


In [4]:
carts = carts.sort_values(['customer_id', 't_dat'])

In [5]:
carts['day_of_week'] = carts['t_dat'].dt.day_of_week

### Ajout de l'intervalle entre deux paniers

In [6]:
carts['day_from_origin'] = (carts['t_dat'] - carts['t_dat'].min()).dt.days

In [7]:
carts['days_from_previous'] = carts['day_from_origin'].diff()

In [8]:
carts['previous_customer_id']=""
carts['previous_customer_id'].iloc[1:] = carts['customer_id'].iloc[0:-1]
carts.loc[carts['customer_id'] != carts['previous_customer_id'], 'days_from_previous'] = 0

/home/treb/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
carts['days_from_previous'] = carts['days_from_previous'].fillna(0).astype(int)
carts = carts.drop('previous_customer_id', 1)

/tmp/ipykernel_21069/2697235294.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  carts = carts.drop('previous_customer_id', 1)


In [11]:
carts.to_pickle('pickles/carts.pkl')
carts.head()

,customer_id,t_dat,sales_channel_id,total,unique_articles,total_articles,week_number,content,day_of_week,day_from_origin,days_from_previous
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,1,0.110119,3,3,90,"[0627759010, 0625548001, 0176209023]",3,98,0
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,2,0.010153,1,1,72,[0697138006],3,224,126
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2,0.050831,1,2,69,[0568601006],5,247,23
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,1,0.033864,2,2,60,"[0745232001, 0607642008]",3,308,61
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,2,0.044051,1,1,52,[0656719005],2,363,55


In [12]:
carts[['customer_id', 't_dat', 'total', 'unique_articles', 'total_articles', 'week_number', 'days_from_previous']].head(5)

,customer_id,t_dat,total,unique_articles,total_articles,week_number,days_from_previous
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,0.110119,3,3,90,0
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,0.010153,1,1,72,126
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,0.050831,1,2,69,23
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,0.033864,2,2,60,61
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,0.044051,1,1,52,55


## Table client

In [27]:
customers = pd.read_pickle('pickles/customers.pkl')

In [43]:
customers['club_member_status'].isna().sum()

6062

In [41]:
customers['club_member_status'].value_counts()

ACTIVE        1272491
PRE-CREATE      92960
LEFT CLUB         467
Name: club_member_status, dtype: int64

#### Nettoyage de la table

In [56]:

 ## fichier 'customer'
   ### Colonne FN
     #### Remplacement des NaN par 0
customers['FN'] = customers['FN'].fillna(0)

   ### Colonne Active
     #### Remplacement des NaN par 0
customers['Active'] = customers['Active'].fillna(0)

   ### Colonne Club member status
     #### Remplacement des NaN par NON ACTIVE
customers['club_member_status'] = customers['club_member_status'].fillna('NON ACTIVE')
customers[customers['club_member_status']=='NON ACTIVE'] 

   ### Colonne Fashion News Frequency
     #### Option 1: Remplacement NaN par None
customers['fashion_news_frequency'].fillna('None', inplace = True)
     #### Correction d'un label écrit avec deux casses différentes
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace({'NONE': 'None'})

   ### Colonne age
     #### Option 1: Remplacement NaN par None
#customers['age'] = customers['age'].fillna('NONE')
     #### Option 2: Remplacement par la moyenne
#customers['age'] = customers['age'].fillna(customers['age'].mean())
     #### Option 3: Remplacement par zéro
customers['age'] = customers['age'].fillna(0)


#### Ajout de features liées aux achats

In [58]:
carts = pd.read_pickle('pickles/carts.pkl')

carts_customers = carts.groupby('customer_id', as_index = False).agg(
    average_cart_articles = ("unique_articles", "mean"),
    average_cart_price = ("total", "mean"),
    total_carts = ('customer_id', 'count'),
    total_articles = ("total_articles", 'sum'),
    total_price = ("total", 'sum'),
)
carts_customers.head()

,customer_id,average_cart_articles,average_cart_price,total_carts,total_articles,total_price
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,1.900000,0.054393,10,21,0.543932
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,3.250000,0.100510,24,86,2.412237
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2.142857,0.086646,7,18,0.606525
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2.000000,0.060983,1,2,0.060983
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2.166667,0.078282,6,13,0.469695


In [59]:
# Intervalle moyen entre chaque panier
carts_intervals = carts[carts['days_from_previous'] != 0].groupby('customer_id', as_index = False).agg(
    average_cart_interval = ('days_from_previous', 'mean')
)
carts_intervals['average_cart_interval'] = carts_intervals['average_cart_interval'] / 7
carts_intervals.head()

,customer_id,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,17.285714
3,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,19.142857
4,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,33.428571


In [60]:
# Ajout des features sur la table des clients.
customers = customers.merge(carts_customers, on = 'customer_id', how = 'left')
customers = customers.merge(carts_intervals, on = 'customer_id', how = 'left')

# Remplissage des valeurs manquantes suites aux jointures.
customers.fillna(0, inplace = True)

In [74]:
# Typer les champs entiers en int32.
customers[['FN', 'Active', 'age', 'total_carts', 'total_articles']] = customers[['FN', 'Active', 'age', 'total_carts', 'total_articles']].astype('int16')

# Typer les champs de catégories en "categorical"
customers[['club_member_status', 'fashion_news_frequency', 'postal_code']] = customers[['club_member_status', 'fashion_news_frequency', 'postal_code']].astype('category')

In [76]:
customers.to_pickle('pickles/customers_clean.pkl')
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,average_cart_articles,average_cart_price,total_carts,total_articles,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.900000,0.054393,10,21,0.543932,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,None,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,3.250000,0.100510,24,86,2.412237,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,None,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,2.142857,0.086646,7,18,0.606525,17.285714
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,None,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,2.000000,0.060983,1,2,0.060983,0.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,2.166667,0.078282,6,13,0.469695,19.142857


In [77]:

customers.iloc[:, :7].head(5)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,None,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,None,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,None,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [79]:
customers[['customer_id', 'average_cart_articles',
       'average_cart_price', 'total_carts', 'total_articles', 'total_price',
       'average_cart_interval']].head(5)

,customer_id,average_cart_articles,average_cart_price,total_carts,total_articles,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,1.900000,0.054393,10,21,0.543932,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,3.250000,0.100510,24,86,2.412237,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2.142857,0.086646,7,18,0.606525,17.285714
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2.000000,0.060983,1,2,0.060983,0.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2.166667,0.078282,6,13,0.469695,19.142857


## Table Articles

In [27]:
articles = pd.read_pickle('pickles/articles.pkl')

#### Nettoyage de la table

In [28]:
# Étape 3: Gestion des NaN
articles['detail_desc'].fillna('', inplace = True)

#### Features issues des transactions

In [ ]:
transactions = pd.read_pickle('pickles/transactions_clean.pkl')

transaction_articles = transactions.groupby('article_id', as_index = False).agg(
    total_purchases = ('quantity', 'sum'),
    average_quantity = ('quantity', 'mean'),
    average_price = ('price', 'mean')
)

In [29]:
articles = articles.merge(transaction_articles, on = 'article_id', how = 'left')

# Suppression des articles pour lesquels aucune transaction n'existe.
articles.dropna(subset = ['total_purchases'], axis = 0, inplace = True)

#### Ajout d'une colonne has_image (True / False)

In [30]:
# Chargement de la liste des fichiers images
# (Obtenu par la commande `find -name  *.jpg > images.csv` sous linux) 

images = pd.read_csv('csv/images.csv')
images['article_id'] = images['file_path'].apply(
    lambda x: 
        x.split('/')[2].split('.jpg')[0]
).astype(str)
images = images.sort_values('article_id')
images.head()

,file_path,article_id
73653,./010/0108775015.jpg,0108775015
73652,./010/0108775044.jpg,0108775044
73651,./010/0108775051.jpg,0108775051
16811,./011/0110065001.jpg,0110065001
16810,./011/0110065002.jpg,0110065002


In [31]:
# Fusion des deux tableaux en Left join
article_images = articles.merge(images, on ="article_id", how = 'left')

In [ ]:
article_images['has_image'] = article_images['file_path'].isna().apply(lambda x: not x)
articles = article_images.drop("file_path", 1)

#### Finalisation

In [33]:
# Suppression des catégories au format numérique
columns_to_drop = ['product_type_no',
       'graphical_appearance_no',
       'colour_group_code',
       'perceived_colour_value_id',
       'perceived_colour_master_id',
       'department_no',
       'index_group_no',
       'section_no',
       'garment_group_no']

articles.drop(columns = columns_to_drop, axis = 1, inplace = True)

# Typage des catégories 
categories = ['product_type_name', 'product_group_name', 
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name',
       'perceived_colour_master_name',
       'department_name', 'index_name',
       'index_group_name', 'section_name',
       'garment_group_name']

articles[categories] = articles[categories].astype('category')

# Typage des champs entiers
articles['total_purchases'] = articles['total_purchases'].astype('int32')

In [34]:
# Sauvegarde finale
articles.to_pickle('pickles/articles_clean.pkl')
articles.head()

,article_id,product_code,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,...,index_name,index_group_name,section_no,section_name,garment_group_name,detail_desc,total_purchases,average_quantity,average_price,has_image
0,0108775015,0108775,Strap top,Vest top,Garment Upper body,Solid,Black,Dark,Black,Jersey Basic,...,Ladieswear,Ladieswear,16,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,10841,1.438752,0.008109,True
1,0108775044,0108775,Strap top,Vest top,Garment Upper body,Solid,White,Light,White,Jersey Basic,...,Ladieswear,Ladieswear,16,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,7250,1.279562,0.008093,True
2,0108775051,0108775,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Dusty Light,White,Jersey Basic,...,Ladieswear,Ladieswear,16,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,215,1.214689,0.004982,True
3,0110065001,0110065,OP T-shirt (Idro),Bra,Underwear,Solid,Black,Dark,Black,Clean Lingerie,...,Lingeries/Tights,Ladieswear,61,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",1044,1.063136,0.020055,True
4,0110065002,0110065,OP T-shirt (Idro),Bra,Underwear,Solid,White,Light,White,Clean Lingerie,...,Lingeries/Tights,Ladieswear,61,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",539,1.118257,0.018087,True


In [35]:
articles.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_name',
       'product_group_name', 'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_code', 'index_name', 'index_group_name',
       'section_no', 'section_name', 'garment_group_name', 'detail_desc',
       'total_purchases', 'average_quantity', 'average_price', 'has_image'],
      dtype='object')

In [36]:
articles[['article_id', 'product_code', 'prod_name', 'product_type_name',
       'product_group_name', 'garment_group_name',
       'total_purchases', 'average_quantity', 'average_price', 'has_image']].head()

,article_id,product_code,prod_name,product_type_name,product_group_name,garment_group_name,total_purchases,average_quantity,average_price,has_image
0,0108775015,0108775,Strap top,Vest top,Garment Upper body,Jersey Basic,10841,1.438752,0.008109,True
1,0108775044,0108775,Strap top,Vest top,Garment Upper body,Jersey Basic,7250,1.279562,0.008093,True
2,0108775051,0108775,Strap top (1),Vest top,Garment Upper body,Jersey Basic,215,1.214689,0.004982,True
3,0110065001,0110065,OP T-shirt (Idro),Bra,Underwear,"Under-, Nightwear",1044,1.063136,0.020055,True
4,0110065002,0110065,OP T-shirt (Idro),Bra,Underwear,"Under-, Nightwear",539,1.118257,0.018087,True


## Jointure Transactions - Customers - Articles
Table générée à des fins d'analyse.

In [42]:
transactions = pd.read_pickle("pickles/transactions_clean.pkl")
customers = pd.read_pickle("pickles/customers_clean.pkl")
articles = pd.read_pickle("pickles/articles_clean.pkl")

transactions = transactions.sample(frac = 0.1)

In [43]:
transactions = transactions.merge(customers, on="customer_id", how="left")

In [44]:
transactions = transactions.merge(articles, on="article_id", how="left")

In [45]:
transactions.to_pickle("pickles/transactions_all_infos.pkl")

In [47]:
transactions.head(2)

,t_dat,customer_id,article_id,price,sales_channel_id,quantity,week_number,FN,Active,club_member_status,...,index_name,index_group_name,section_no,section_name,garment_group_name,detail_desc,total_purchases,average_quantity,average_price,has_image
0,2019-03-25,251fba075eddd3298fd9a2ec2465529faabc62f9d09ff1...,0632982033,0.014712,1,1,78,1,1,ACTIVE,...,Ladieswear,Ladieswear,15,Womens Everyday Collection,Jersey Fancy,"Short-sleeved, round-necked T-shirt in jersey ...",1624,1.037700,0.014132,True
1,2019-08-23,98d4e12ceb769ae0bf17580dadee98a06aaeae405dd7fc...,0703207004,0.013542,1,1,56,1,1,ACTIVE,...,Lingeries/Tights,Ladieswear,61,Womens Lingerie,"Under-, Nightwear",Thong briefs in microfibre with a wide lace tr...,919,1.047891,0.015267,True


## Jointure Carts - Customers

Table générée à des fins d'analyse.

In [1]:
import pandas as pd
carts = pd.read_pickle("pickles/carts.pkl")
customers = pd.read_pickle("pickles/customers_clean.pkl")

In [2]:
carts = carts.merge(customers, on="customer_id", how="left")

In [3]:
carts.to_pickle("pickles/carts_all_infos.pkl")
carts.head()

,customer_id,t_dat,sales_channel_id,total,unique_articles,total_articles_x,week_number,content,day_of_week,day_from_origin,...,club_member_status,fashion_news_frequency,age,postal_code,average_cart_articles,average_cart_price,total_carts,total_articles_y,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,1,0.110119,3,3,90,"[0627759010, 0625548001, 0176209023]",3,98,...,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10,21,0.543932,9.809524
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,2,0.010153,1,1,72,[0697138006],3,224,...,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10,21,0.543932,9.809524
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2,0.050831,1,2,69,[0568601006],5,247,...,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10,21,0.543932,9.809524
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,1,0.033864,2,2,60,"[0745232001, 0607642008]",3,308,...,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10,21,0.543932,9.809524
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,2,0.044051,1,1,52,[0656719005],2,363,...,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10,21,0.543932,9.809524
